In [28]:
import numpy as np

In [60]:
class Tensor:
    def __init__(self,data,autograd=False,creators=None,creation_ops=None,id=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_ops = creation_ops
        self.autograd = autograd
        self.children = {}
        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1
        
        
        self.grad = None
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True
    
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_ops == "add"):
                    self.creators[0].backward(self.grad,self)
                    self.creators[1].backward(self.grad,self)
    def __add__(self,other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_ops="add")
        return Tensor(self.data + other.data,creators=[self,other],creation_ops="add")
    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

In [61]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([2,2,2,2,2])))

print(b.grad.data == np.array([2,2,2,2,2]))

[False False False False False]


In [67]:
d.children

{63638: 0}

In [39]:
z.creators

[array([1, 1, 1, 1, 1]), array([2, 2, 2, 2, 2])]

In [17]:
z.backward(Tensor(np.array([1,1,1,1,1])))

In [19]:
b.grad

array([1, 1, 1, 1, 1])

In [36]:
y  = z + b

In [22]:
y.creators

[array([3, 3, 3, 3, 3]), array([2, 2, 2, 2, 2])]

In [26]:
a.grad

array([1, 1, 1, 1, 1])

In [42]:
a.children.items()

dict_items([(64833, 1), (89815, 1)])

In [41]:
c = a + y